In [ ]:
# default_exp model

# Model

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import pytorch_lightning as pl

from transformers import AdamW, get_cosine_with_hard_restarts_schedule_with_warmup as cosine_restart_schedule

In [ ]:
# export
# Code from: https://github.com/huggingface/transformers/blob/master/examples/research_projects/codeparrot/scripts/codeparrot_training.py#L113
def get_grouped_params(model, weight_decay, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n, p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
    return [
        {"params": params_with_wd, "weight_decay": weight_decay},
        {"params": params_without_wd, "weight_decay": 0.0},
    ]

In [ ]:
# export
class RetroFitModelWrapper(pl.LightningModule):
    def __init__(
        self,
        model,
        weight_decay=0.1,
        lr=5e-4,
        num_warmup_steps=2_000,
        freeze_decoder=True
    ):
        super().__init__()
        self.model = model
        self.weight_decay = weight_decay
        self.lr = lr
        self.num_warmup_steps = num_warmup_steps
        self.freeze_decoder = freeze_decoder

        self.save_hyperparameters()

    def training_step(self, batch, batch_idx):
        loss = self.model(input_ids=batch["retrieved_input_ids"], labels=batch["input_ids"]).loss

        self.log("trn_loss", loss, on_step=True, on_epoch=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.model(input_ids=batch["retrieved_input_ids"], labels=batch["input_ids"]).loss

        self.log("val_loss", loss, on_epoch=True, logger=True)
        return loss
    
    def configure_optimizers(self):
        # Prepare the optimizer and learning rate scheduler
        param_model = self.model if not self.freeze_decoder else self.model.encoder
        optimizer = AdamW(get_grouped_params(param_model, self.weight_decay), lr=self.lr)
        lr_scheduler = cosine_restart_schedule(
            optimizer=optimizer,
            num_warmup_steps=self.num_warmup_steps,
            num_training_steps=self.total_steps(),
        )
        return [optimizer], [lr_scheduler]
    
    def total_steps(self) -> int:
        """The number of total training steps that will be run. Used for lr scheduler purposes."""
        dataset_size = len(self.trainer.datamodule.train_dataloader())
        num_devices = max(1, self.trainer.gpus)  # TODO: consider num_tpu_cores
        effective_batch_size = self.trainer.datamodule.batch_size * num_devices
        return (dataset_size / effective_batch_size) * self.trainer.max_epochs

    def generate(self, retrieved_examples, encoder_tokenizer, decoder_tokenizer, skip_special_tokens=True):
        retrieved = [encoder_tokenizer.eos_token.join(r) for r in retrieved_examples]
        input_ids = encoder_tokenizer(retrieved, return_tensors="pt").input_ids
        generated = self.model.generate(input_ids, decoder_start_token_id=self.model.config.decoder.bos_token_id)

        return decoder_tokenizer.decode(generated, skip_special_tokens=skip_special_tokens)

In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()

Converted 00_data.ipynb.
Converted 01_model.ipynb.
Converted 02_tutorial.ipynb.
Converted index.ipynb.
